In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.datasets import fashion_mnist, mnist
from tensorflow.keras.backend import clear_session
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [5]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [18]:
(X_train_org, y_train_org), (X_test_org, y_test_org) = fashion_mnist.load_data()

X_train_org = X_train_org.reshape(-1, 28, 28, 1)
X_test_org = X_test_org.reshape(-1, 28, 28, 1)

X_train_org = X_train_org.astype('float32')
X_test_org = X_test_org.astype('float32')
X_train_org /= 255
X_test_org /= 255

In [78]:
# 1,2,5,10 conv2d
# 5, 10, 15 epok
# 100, 1000 wag
# 2500, 5000, 10000 sampli, 0.8 ratio

def gen_3a_svm():
    for ratio in np.arange(0.025, 1.0, 0.025):
        train_samples = int(10000 * ratio)
        test_samples = 10000 - train_samples
        X_train, y_train = resample(X_train_org, y_train_org, random_state=0, n_samples=train_samples)
        X_test, y_test = resample(X_test_org, y_test_org, random_state=0, n_samples=test_samples)
        
        svc = SVC(C=5.0, kernel='rbf')
        svc.fit(X_train.reshape(-1, 784), y_train)
        y_pred = svc.predict(X_test.reshape(-1, 784))
        accuracy = accuracy_score(y_test, y_pred)
        print(ratio)
        yield ratio, accuracy, 'SVM'

results_3a_svm = pd.DataFrame(gen_3a_svm(), columns=['ratio', 'accuracy', 'name'])

0.025
0.05
0.07500000000000001
0.1
0.125
0.15
0.17500000000000002
0.2
0.225
0.25
0.275
0.30000000000000004
0.32500000000000007
0.35000000000000003
0.37500000000000006
0.4
0.42500000000000004
0.45000000000000007
0.47500000000000003
0.5
0.525
0.55
0.5750000000000001
0.6000000000000001
0.6250000000000001
0.65
0.675
0.7000000000000001
0.7250000000000001
0.7500000000000001
0.775
0.8
0.8250000000000001
0.8500000000000001
0.8750000000000001
0.9
0.925
0.9500000000000001
0.9750000000000001


In [114]:
def number_of_params(layers):
    Sequential(layers).summary()
    return Sequential(layers).count_params()

def evaluate_cnn(layers, ratio, epochs):
    train_samples = int(10000 * ratio)
    test_samples = 10000 - train_samples
    X_train, y_train = resample(X_train_org, y_train_org, random_state=0, n_samples=train_samples)
    X_test, y_test = resample(X_test_org, y_test_org, random_state=0, n_samples=test_samples)

    clear_session()
    model = Sequential(layers)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    return accuracy

def gen_3a_cnn(layers, name):
    for ratio in np.arange(0.025, 1.0, 0.025):
        accuracy = evaluate_cnn(layers, ratio, 10)
        yield ratio, accuracy, name


model1 = [
    Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
]

model2 = [
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
]

model5 = [
    Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
]

model10 = [
    Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
]

results_3a_1  = pd.DataFrame(gen_3a_cnn(model1,  'CNN 1'))
results_3a_2  = pd.DataFrame(gen_3a_cnn(model2,  'CNN 2'))
results_3a_5  = pd.DataFrame(gen_3a_cnn(model5,  'CNN 5'))
results_3a_10 = pd.DataFrame(gen_3a_cnn(model10, 'CNN 10'))

Train on 250 samples, validate on 9750 samples
Epoch 1/10
250/250 [==============================] - 1s 4ms/sample - loss: 1.9365 - accuracy: 0.3200 - val_loss: 1.5902 - val_accuracy: 0.5207
Epoch 2/10
250/250 [==============================] - 1s 3ms/sample - loss: 1.1908 - accuracy: 0.6760 - val_loss: 1.0872 - val_accuracy: 0.6218
Epoch 3/10
250/250 [==============================] - 1s 2ms/sample - loss: 0.7998 - accuracy: 0.7520 - val_loss: 0.8905 - val_accuracy: 0.7127
Epoch 4/10
250/250 [==============================] - 1s 2ms/sample - loss: 0.6381 - accuracy: 0.8280 - val_loss: 0.7989 - val_accuracy: 0.7264
Epoch 5/10
250/250 [==============================] - 1s 2ms/sample - loss: 0.5215 - accuracy: 0.8440 - val_loss: 0.8378 - val_accuracy: 0.7143
Epoch 6/10
250/250 [==============================] - 1s 3ms/sample - loss: 0.4536 - accuracy: 0.8520 - val_loss: 0.6992 - val_accuracy: 0.7513
Epoch 7/10
250/250 [==============================] - 1s 2ms/sample - loss: 0.3835 - accu

1500/1500 [==============================] - 1s 399us/sample - loss: 0.0488 - accuracy: 0.9900 - val_loss: 0.6026 - val_accuracy: 0.8312
Epoch 6/10
1500/1500 [==============================] - 1s 389us/sample - loss: 0.0372 - accuracy: 0.9940 - val_loss: 0.5962 - val_accuracy: 0.8372
Epoch 7/10
1500/1500 [==============================] - 1s 386us/sample - loss: 0.0355 - accuracy: 0.9927 - val_loss: 0.6265 - val_accuracy: 0.8315
Epoch 8/10
1500/1500 [==============================] - 1s 431us/sample - loss: 0.0202 - accuracy: 0.9980 - val_loss: 0.6886 - val_accuracy: 0.8279
Epoch 9/10
1500/1500 [==============================] - 1s 391us/sample - loss: 0.0202 - accuracy: 0.9993 - val_loss: 0.6345 - val_accuracy: 0.8321
Epoch 10/10
1500/1500 [==============================] - 1s 380us/sample - loss: 0.0139 - accuracy: 1.0000 - val_loss: 0.6693 - val_accuracy: 0.8351
Train on 1750 samples, validate on 8250 samples
Epoch 1/10
1750/1750 [==============================] - 1s 551us/sample - 

Epoch 9/10
2750/2750 [==============================] - 1s 218us/sample - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.7807 - val_accuracy: 0.8549
Epoch 10/10
2750/2750 [==============================] - 1s 217us/sample - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.8024 - val_accuracy: 0.8527
Train on 3000 samples, validate on 7000 samples
Epoch 1/10
3000/3000 [==============================] - 1s 374us/sample - loss: 0.0857 - accuracy: 0.9800 - val_loss: 0.7349 - val_accuracy: 0.8489
Epoch 2/10
3000/3000 [==============================] - 1s 245us/sample - loss: 0.0402 - accuracy: 0.9880 - val_loss: 0.7740 - val_accuracy: 0.8464
Epoch 3/10
3000/3000 [==============================] - 1s 233us/sample - loss: 0.0191 - accuracy: 0.9970 - val_loss: 0.7403 - val_accuracy: 0.8540
Epoch 4/10
3000/3000 [==============================] - 1s 222us/sample - loss: 0.0159 - accuracy: 0.9943 - val_loss: 0.8095 - val_accuracy: 0.8453
Epoch 5/10
3000/3000 [==============================] - 1s 206u

4250/4250 [==============================] - 1s 167us/sample - loss: 0.0196 - accuracy: 0.9953 - val_loss: 0.7757 - val_accuracy: 0.8574
Epoch 3/10
4250/4250 [==============================] - 1s 146us/sample - loss: 0.0085 - accuracy: 0.9988 - val_loss: 0.7870 - val_accuracy: 0.8537
Epoch 4/10
4250/4250 [==============================] - 1s 148us/sample - loss: 0.0085 - accuracy: 0.9979 - val_loss: 0.7803 - val_accuracy: 0.8600
Epoch 5/10
4250/4250 [==============================] - 1s 160us/sample - loss: 0.0083 - accuracy: 0.9981 - val_loss: 0.7810 - val_accuracy: 0.8597
Epoch 6/10
4250/4250 [==============================] - 1s 162us/sample - loss: 0.0125 - accuracy: 0.9967 - val_loss: 0.7744 - val_accuracy: 0.8605
Epoch 7/10
4250/4250 [==============================] - 1s 196us/sample - loss: 0.0043 - accuracy: 0.9993 - val_loss: 0.7834 - val_accuracy: 0.8652
Epoch 8/10
4250/4250 [==============================] - 1s 184us/sample - loss: 9.1468e-04 - accuracy: 1.0000 - val_loss: 0

5500/5500 [==============================] - 1s 120us/sample - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.8716 - val_accuracy: 0.8624
Epoch 6/10
5500/5500 [==============================] - 1s 118us/sample - loss: 0.0046 - accuracy: 0.9980 - val_loss: 0.8835 - val_accuracy: 0.8647
Epoch 7/10
5500/5500 [==============================] - 1s 122us/sample - loss: 0.0098 - accuracy: 0.9971 - val_loss: 0.8804 - val_accuracy: 0.8611
Epoch 8/10
5500/5500 [==============================] - 1s 119us/sample - loss: 0.0040 - accuracy: 0.9993 - val_loss: 0.9287 - val_accuracy: 0.8618
Epoch 9/10
5500/5500 [==============================] - 1s 121us/sample - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.9147 - val_accuracy: 0.8640
Epoch 10/10
5500/5500 [==============================] - 1s 119us/sample - loss: 5.4642e-04 - accuracy: 1.0000 - val_loss: 0.9226 - val_accuracy: 0.8638
Train on 5750 samples, validate on 4250 samples
Epoch 1/10
5750/5750 [==============================] - 1s 184us/sampl

...

4250/4250 [==============================] - 1s 312us/sample - loss: 0.0207 - accuracy: 0.9951 - val_loss: 0.9973 - val_accuracy: 0.8303
Epoch 4/10
4250/4250 [==============================] - 1s 304us/sample - loss: 0.0180 - accuracy: 0.9946 - val_loss: 1.2109 - val_accuracy: 0.8270
Epoch 5/10
4250/4250 [==============================] - 1s 304us/sample - loss: 0.0259 - accuracy: 0.9911 - val_loss: 0.9612 - val_accuracy: 0.8172
Epoch 6/10
4250/4250 [==============================] - 1s 309us/sample - loss: 0.0322 - accuracy: 0.9908 - val_loss: 1.2159 - val_accuracy: 0.8195
Epoch 7/10
4250/4250 [==============================] - 1s 306us/sample - loss: 0.0216 - accuracy: 0.9918 - val_loss: 1.3294 - val_accuracy: 0.8243
Epoch 8/10
4250/4250 [==============================] - 1s 304us/sample - loss: 0.0250 - accuracy: 0.9913 - val_loss: 1.1016 - val_accuracy: 0.8252
Epoch 9/10
4250/4250 [==============================] - 1s 306us/sample - loss: 0.0553 - accuracy: 0.9824 - val_loss: 1.143

Epoch 7/10
5500/5500 [==============================] - 1s 265us/sample - loss: 0.0214 - accuracy: 0.9936 - val_loss: 1.2427 - val_accuracy: 0.8289
Epoch 8/10
5500/5500 [==============================] - 1s 268us/sample - loss: 0.0210 - accuracy: 0.9951 - val_loss: 1.2454 - val_accuracy: 0.8376
Epoch 9/10
5500/5500 [==============================] - 1s 266us/sample - loss: 0.0064 - accuracy: 0.9971 - val_loss: 1.6620 - val_accuracy: 0.8369
Epoch 10/10
5500/5500 [==============================] - 1s 266us/sample - loss: 0.0224 - accuracy: 0.9931 - val_loss: 1.1396 - val_accuracy: 0.8289
Train on 5750 samples, validate on 4250 samples
Epoch 1/10
5750/5750 [==============================] - 2s 421us/sample - loss: 0.0432 - accuracy: 0.9911 - val_loss: 1.0802 - val_accuracy: 0.8421
Epoch 2/10
5750/5750 [==============================] - 1s 251us/sample - loss: 0.0120 - accuracy: 0.9969 - val_loss: 1.1942 - val_accuracy: 0.8327
Epoch 3/10
5750/5750 [==============================] - 1s 249u

6750/6750 [==============================] - 2s 234us/sample - loss: 0.0105 - accuracy: 0.9979 - val_loss: 1.3958 - val_accuracy: 0.8397
Train on 7000 samples, validate on 3000 samples
Epoch 1/10
7000/7000 [==============================] - 3s 364us/sample - loss: 0.0437 - accuracy: 0.9903 - val_loss: 1.1465 - val_accuracy: 0.8363
Epoch 2/10
7000/7000 [==============================] - 2s 223us/sample - loss: 0.0110 - accuracy: 0.9961 - val_loss: 1.4466 - val_accuracy: 0.8487
Epoch 3/10
7000/7000 [==============================] - 2s 241us/sample - loss: 0.0013 - accuracy: 0.9997 - val_loss: 1.5341 - val_accuracy: 0.8530
Epoch 4/10
7000/7000 [==============================] - 2s 245us/sample - loss: 0.0272 - accuracy: 0.9927 - val_loss: 1.2703 - val_accuracy: 0.8323
Epoch 5/10
7000/7000 [==============================] - 2s 228us/sample - loss: 0.0334 - accuracy: 0.9900 - val_loss: 1.3942 - val_accuracy: 0.8347
Epoch 6/10
7000/7000 [==============================] - 2s 221us/sample - l

8250/8250 [==============================] - 2s 200us/sample - loss: 0.0272 - accuracy: 0.9914 - val_loss: 1.2635 - val_accuracy: 0.8383
Epoch 4/10
8250/8250 [==============================] - 2s 201us/sample - loss: 0.0221 - accuracy: 0.9936 - val_loss: 1.4035 - val_accuracy: 0.8406
Epoch 5/10
8250/8250 [==============================] - 2s 202us/sample - loss: 0.0207 - accuracy: 0.9938 - val_loss: 1.2532 - val_accuracy: 0.8394
Epoch 6/10
8250/8250 [==============================] - 2s 202us/sample - loss: 0.0068 - accuracy: 0.9973 - val_loss: 1.3872 - val_accuracy: 0.8463
Epoch 7/10
8250/8250 [==============================] - 2s 205us/sample - loss: 0.0372 - accuracy: 0.9909 - val_loss: 1.2304 - val_accuracy: 0.8463
Epoch 8/10
8250/8250 [==============================] - 2s 202us/sample - loss: 0.0117 - accuracy: 0.9962 - val_loss: 1.5214 - val_accuracy: 0.8451
Epoch 9/10
8250/8250 [==============================] - 2s 205us/sample - loss: 0.0088 - accuracy: 0.9973 - val_loss: 1.466

Epoch 7/10
9500/9500 [==============================] - 3s 298us/sample - loss: 0.0249 - accuracy: 0.9937 - val_loss: 1.4849 - val_accuracy: 0.8460
Epoch 8/10
9500/9500 [==============================] - 3s 351us/sample - loss: 0.0221 - accuracy: 0.9940 - val_loss: 1.6357 - val_accuracy: 0.8700
Epoch 9/10
9500/9500 [==============================] - 3s 293us/sample - loss: 0.0111 - accuracy: 0.9966 - val_loss: 1.7545 - val_accuracy: 0.8500
Epoch 10/10
9500/9500 [==============================] - 3s 298us/sample - loss: 0.0268 - accuracy: 0.9921 - val_loss: 1.6980 - val_accuracy: 0.8560
Train on 9750 samples, validate on 250 samples
Epoch 1/10
9750/9750 [==============================] - 4s 453us/sample - loss: 0.0376 - accuracy: 0.9923 - val_loss: 1.1963 - val_accuracy: 0.8720
Epoch 2/10
9750/9750 [==============================] - 4s 367us/sample - loss: 0.0119 - accuracy: 0.9963 - val_loss: 1.4981 - val_accuracy: 0.8760
Epoch 3/10
9750/9750 [==============================] - 4s 401us

In [118]:
results_3a_1.columns = results_3a_2.columns = results_3a_5.columns = results_3a_10.columns = ['ratio', 'accuracy', 'name']

concat_1  = pd.concat((results_3a_svm, results_3a_1))
concat_2  = pd.concat((results_3a_svm, results_3a_2))
concat_5  = pd.concat((results_3a_svm, results_3a_5))
concat_10 = pd.concat((results_3a_svm, results_3a_10))

def plot(data, filename):
    ax = sns.lmplot(x='ratio', y='accuracy', hue='name', data=data)
    ax.set(xlim=(0.08, 0.92))
    ax.savefig(filename)
    
plot(results_3a_1,  'cnn_1.svg')
plot(concat_2,  'cnn_2.svg')
plot(concat_5,  'cnn_5.svg')
plot(concat_10, 'cnn_10.svg')

KeyError: "None of [Index(['accuracy', 'name', 'ratio'], dtype='object')] are in the [columns]"

In [17]:
model = Sequential([
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)

print({'accuracy': accuracy, 'loss': loss})

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
25952/60000 [===========>..................] - ETA: 3s - loss: 0.4932 - accuracy: 0.8219

KeyboardInterrupt: 